Data PreProcessing

In [115]:
import spacy
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import contractions

In [116]:
file_path = "master-gdc-gdcdatasets-2020446966_en-2020446966_en\wdl_data_en.csv"
df = pd.read_csv(file_path)

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\User\AppData\Local\Temp\ipykernel_38876\316878166.py:1: SyntaxWarning: invalid escape sequence '\w'
  file_path = "master-gdc-gdcdatasets-2020446966_en-2020446966_en\wdl_data_en.csv"
C:\Users\User\AppData\Local\Temp\ipykernel_38876\316878166.py:2: DtypeWarning: Columns (23,30,31,36,37,38,40,44,45,46,48,52,53,54,55,56,57,58,60,61,64,65,66,69,70,72,73,74,75,77,79,80,81,82,83,84,85,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [117]:
# Load spaCy's English NER model
nlp = spacy.load("en_core_web_sm")

In [118]:
# Count columns before dropping
initial_columns = df.shape[1]

In [119]:
threshold = 75
df = df.loc[:, df.isnull().mean() * 100 < threshold]

In [120]:
# Count columns after dropping
final_columns = df.shape[1]

In [121]:
df= df.drop(["wdl_id", "collection", "wdl_url", "handle","title_in_original_language", "Date Created", "Dewey Decimal Code", "Additional Subjects", "Physical Description", "Note", "Type of Item", "Language", "Place of Publication"], axis=1)

In [122]:
# Display results
print(f"Columns before dropping: {initial_columns}")
print(f"Columns after dropping: {final_columns}")
print(f"Columns removed: {initial_columns - final_columns}")

Columns before dropping: 102
Columns after dropping: 19
Columns removed: 83


In [123]:

# Function to remove accents
def remove_accents(text):
    if isinstance(text, str):
        return unidecode(text)
    return text

# Apply to all text columns
df = df.applymap(remove_accents)

C:\Users\User\AppData\Local\Temp\ipykernel_38876\2058031408.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(remove_accents)


In [124]:
# Function to clean and preprocess text
def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = contractions.fix(text)  # Expand contractions
    text = text.lower() 
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    doc = nlp(text)
    # Remove stopwords and lemmatize
    clean_text = " ".join([token.lemma_ for token in doc])

    return clean_text

In [125]:
# Randomly pick 5000 rows
df = df.sample(n=5000, random_state=1)

In [126]:
# Save cleaned data
cleaned_file_path = "master-gdc-gdcdatasets-2020446966_en-2020446966_en/output1.csv"
df.to_csv(cleaned_file_path, index=False)